# Simple model

This is the code to train a simple model using keras. This seems to be unefficient, making a very complex and big CNN. However with no no more then 3500 images it converges well to almost 100% accuracy (on evaluation). Still only using, one model per person. Using only one type of attack.

The need of a more robust method would be good. A model that can generilize from multiple people and corectly discriminate live faces from spoofed ones, form different attacks. That would require more and better models of style tranfer.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras import callbacks
from keras.regularizers import l2
import pickle
import numpy as np
import datetime
import os
import random
now_datetime  = datetime.datetime.now()
NAME = f"#Face_spoofing300x2_{now_datetime.day:02d}{now_datetime.month:02d}{now_datetime.year}_{now_datetime.hour:02d}{now_datetime.minute:02d}"
dir_pickle = "database_serialized"
dir_models_save = "models"
ignore = []
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


Using TensorFlow backend.


## Model format
This is a simple CNN, that is made of a convolutional and a dense layer. below you choose the numbers of filters of each concolutional layer, and the numbers of neurons of each dense layer

In [2]:
'''
format1 = [[80,140,320],[80,140,320]]
format2 = [[400,300,200],[400,300,200]]
'''
format3 = [[80,120,300],[400]]
NN_formats = [format3]



## The model
The model creaeted here is enought for an almost ideal (acc = 98%) spoof detection model, as expected from a liveness detection methods that include deep learning as shown on many [studies](https://www.emerald.com/insight/content/doi/10.1108/SR-08-2015-0136/full/html) before.

However the artifical creation of spoofed images, may reduce its performance. For 2 different reasons:
1. Non perfect spoofed images. The creating of a model that can generate an spoofed image relayes much on the single image and the trainning data used to create such model. If both are not manage correctly, the creation of bad models may occur. Not to mention that some types of attacks are really hard to imitate e.g. print attacks.
2. Reduced numbers of data. There isn't much data online for this type of problem, and it to the creation of individuals models for each person may prove unresenable, for each person must have minutes (about two) of video of their faces, for a better conversion of the CNN.

For those reasons, it would be needed a deep leraning method that requires less data, to generalize. Something that could be accuired by [this](https://www.sciencedirect.com/science/article/pii/S1047320318301044) method.

In [3]:
def load_data(person):   
    pickle_path =os.path.join(dir_pickle,person)
    pickle_in = open(os.path.join(pickle_path,f"X{person}.pickle"),"rb")
    X = pickle.load(pickle_in)
    pickle_in.close()
    pickle_in = open(os.path.join(pickle_path,f"y{person}.pickle"),"rb")
    y = pickle.load(pickle_in)
    pickle_in.close()
    
    X = X.astype(np.float16)/255.0
    
    return X, y

def create_model(person, pickle_path,format_convolution,format_dense):
    model = Sequential()
    is_first = True
    # Convolutional layers
    for format_c in format_convolution:
        if is_first:
            model.add(layers.Conv2D(format_c,(3,3),input_shape=X.shape[1:]))
            is_first = False
        else:
            model.add(layers.Conv2D(format_c,(3,3)))
        model.add(layers.Activation("relu"))
        model.add(layers.MaxPool2D(pool_size=(3,3)))
    #Flatten the model if needed
    model.add(layers.Flatten())
    
    #Dense layers
    for format_d in format_dense:
        model.add(layers.Dropout(0.12))
        model.add(layers.Dense(format_d,activation="relu",kernel_regularizer=l2(0.002)))
    
    #Output layer
    model.add(layers.Dense(1,activation="sigmoid"))
    
    
    model.compile(loss="binary_crossentropy",
              optimizer = "adam",
              metrics= ["accuracy"])
    
    return model

## Trainning your model
This line trains your model. It is recoomend to use tensorflow-gpu, and to be carefull if your gpu can handle the data, if  you are having *ResourceExhaustedError*, try reducing the **batch size**, or the sizes of your **tesors**.

Keep the eye on your **validation** because that shows your live performance without trainning bias. You may use checkpoints to get the best validation accuracy.

In [4]:
def create_callbacks(person,format_convolution,format_dense):
    if not os.path.isdir(os.path.join(dir_models_save,person)):
        os.mkdir(os.path.join(dir_models_save,person))
    checkpoints = callbacks.ModelCheckpoint(filepath=os.path.join(dir_models_save,person,f"{person}{NAME}{format_convolution}{format_dense}.h5"),
                                                monitor="val_acc",
                                                mode = "max",
                                                verbose = 1,
                                                save_weights_only=False,
                                                save_best_only=True)
    return checkpoints
        
def create_validation(person,val_test_split=0.25):
    ValSet = []
    pickle_path =os.path.join(dir_pickle,person)
    pickle_in = open(os.path.join(pickle_path,f"X{person}Test.pickle"),"rb")
    Xval = pickle.load(pickle_in)
    pickle_in.close()
    pickle_in = open(os.path.join(pickle_path,f"y{person}Test.pickle"),"rb")
    yval = pickle.load(pickle_in)
    pickle_in.close()
    Xval = Xval.astype(np.float16)
    Xval = Xval/255.0
    yval = yval
    print(f"1{Xval.shape}")
    n_exs = int(len(Xval[0]) * val_test_split)
    ValSet = [np.array(Xval[:n_exs-1]),yval[:n_exs-1]]
    print(f"2{ValSet[0].shape}")
    return ValSet
    
    

In [5]:
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow
import gc
# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))

In [6]:
people = os.listdir(dir_pickle)
for person in people:
    if person in ignore:
        continue
    print(f"Loading {person} data")
    X, y = load_data(person)
    ValSet = []
    ValSet = create_validation(person)
    print(f"-->{ValSet[0].shape}")
    print(NN_formats)
    for format_convolution, format_dense in NN_formats:
        # Create the model structure for the person
        print("Creating their model")
        model = create_model(person,os.path.join(dir_pickle,person),format_convolution,format_dense)
        checkpoint = create_callbacks(person,format_convolution,format_dense)
        #Split the serilized test to validation
        #Train model
        print(f"Start {person} trainning")
        model.fit(X,y,batch_size=10,validation_data=ValSet,epochs=4,shuffle=False, callbacks=[checkpoint])
        del model
        reset_keras()
    del X,y,ValSet

Loading 001 data
1(1153, 300, 300, 3)
2(74, 300, 300, 3)
-->(74, 300, 300, 3)
[[[80, 120, 300], [400]]]
Creating their model
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Start 001 trainning
Train on 10635 samples, validate on 74 samples
Epoch 1/4
10630/10635 [============================>.] - ETA: 0s - loss: 0.1541 - acc: 0.9789
Epoch 00001: val_acc improved from -inf to 0.79730, saving model to models\001\001#Face_spoofing300x2_16102019_1727[80, 120, 300][400].h5
10635/10635 [==============================] - 160s 15ms/sample - loss: 0.1541 - acc: 0.9789 - val_loss: 5.6104 - val_acc: 0.7973
Epoch 2/4
10630/10635 [============================>.] - ETA: 0s - loss: 0.0957 - acc: 0.9971
Epoch 00002: val_acc improved from 0.79730 to 1.00000, saving model to models\001\001#Face_spoofing300x2_16102019_1727[80, 120, 300][400].h5

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000002B0F4062318>
Traceback (most recent call last):
  File "E:\Anaconda3\envs\face_spoofing\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


Loading 004 data
1(990, 300, 300, 3)
2(74, 300, 300, 3)
-->(74, 300, 300, 3)
[[[80, 120, 300], [400]]]
Creating their model
Start 004 trainning
Train on 2544 samples, validate on 74 samples
Epoch 1/4
2540/2544 [============================>.] - ETA: 0s - loss: 0.2904 - acc: 0.9646
Epoch 00001: val_acc improved from -inf to 0.82432, saving model to models\004\004#Face_spoofing300x2_16102019_1727[80, 120, 300][400].h5
2544/2544 [==============================] - 37s 14ms/sample - loss: 0.2900 - acc: 0.9646 - val_loss: 1.7364 - val_acc: 0.8243
Epoch 2/4
2540/2544 [============================>.] - ETA: 0s - loss: 0.0376 - acc: 1.0000
Epoch 00002: val_acc improved from 0.82432 to 0.85135, saving model to models\004\004#Face_spoofing300x2_16102019_1727[80, 120, 300][400].h5
2544/2544 [==============================] - 36s 14ms/sample - loss: 0.0376 - acc: 1.0000 - val_loss: 1.4647 - val_acc: 0.8514
Epoch 3/4
2540/2544 [============================>.] - ETA: 0s - loss: 0.1119 - acc: 0.9929
E

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000002B0F4062318>
Traceback (most recent call last):
  File "E:\Anaconda3\envs\face_spoofing\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


Loading 005 data
1(867, 300, 300, 3)
2(74, 300, 300, 3)
-->(74, 300, 300, 3)
[[[80, 120, 300], [400]]]
Creating their model
Start 005 trainning
Train on 2524 samples, validate on 74 samples
Epoch 1/4
2520/2524 [============================>.] - ETA: 0s - loss: 0.2000 - acc: 0.9758
Epoch 00001: val_acc improved from -inf to 0.68919, saving model to models\005\005#Face_spoofing300x2_16102019_1727[80, 120, 300][400].h5
2524/2524 [==============================] - 36s 14ms/sample - loss: 0.1998 - acc: 0.9758 - val_loss: 3.8725 - val_acc: 0.6892
Epoch 2/4
2520/2524 [============================>.] - ETA: 0s - loss: 0.0197 - acc: 0.9992
Epoch 00002: val_acc improved from 0.68919 to 0.81081, saving model to models\005\005#Face_spoofing300x2_16102019_1727[80, 120, 300][400].h5
2524/2524 [==============================] - 35s 14ms/sample - loss: 0.0197 - acc: 0.9992 - val_loss: 0.7422 - val_acc: 0.8108
Epoch 3/4
2520/2524 [============================>.] - ETA: 0s - loss: 0.0575 - acc: 0.9960
E

...

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000002B0F4062318>
Traceback (most recent call last):
  File "E:\Anaconda3\envs\face_spoofing\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


Loading 087 data
1(387, 300, 300, 3)
2(74, 300, 300, 3)
-->(74, 300, 300, 3)
[[[80, 120, 300], [400]]]
Creating their model
Start 087 trainning
Train on 2415 samples, validate on 74 samples
Epoch 1/4
2410/2415 [============================>.] - ETA: 0s - loss: 0.2854 - acc: 0.9531
Epoch 00001: val_acc improved from -inf to 1.00000, saving model to models\087\087#Face_spoofing300x2_16102019_1727[80, 120, 300][400].h5
2415/2415 [==============================] - 36s 15ms/sample - loss: 0.2849 - acc: 0.9532 - val_loss: 0.0149 - val_acc: 1.0000
Epoch 2/4
2410/2415 [============================>.] - ETA: 0s - loss: 0.1291 - acc: 0.9905
Epoch 00002: val_acc did not improve from 1.00000
2415/2415 [==============================] - 33s 14ms/sample - loss: 0.1289 - acc: 0.9905 - val_loss: 0.0916 - val_acc: 0.9865
Epoch 3/4
2410/2415 [============================>.] - ETA: 0s - loss: 0.0198 - acc: 1.0000
Epoch 00003: val_acc did not improve from 1.00000
2415/2415 [==============================]

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000002B0F4062318>
Traceback (most recent call last):
  File "E:\Anaconda3\envs\face_spoofing\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


Loading 088 data
1(379, 300, 300, 3)
2(74, 300, 300, 3)
-->(74, 300, 300, 3)
[[[80, 120, 300], [400]]]
Creating their model
Start 088 trainning
Train on 2377 samples, validate on 74 samples
Epoch 1/4
2370/2377 [============================>.] - ETA: 0s - loss: 0.2517 - acc: 0.9565
Epoch 00001: val_acc improved from -inf to 1.00000, saving model to models\088\088#Face_spoofing300x2_16102019_1727[80, 120, 300][400].h5
2377/2377 [==============================] - 34s 14ms/sample - loss: 0.2510 - acc: 0.9567 - val_loss: 0.0263 - val_acc: 1.0000
Epoch 2/4
2370/2377 [============================>.] - ETA: 0s - loss: 0.0423 - acc: 0.9954
Epoch 00002: val_acc did not improve from 1.00000
2377/2377 [==============================] - 47s 20ms/sample - loss: 0.0422 - acc: 0.9954 - val_loss: 0.0082 - val_acc: 1.0000
Epoch 3/4
2370/2377 [============================>.] - ETA: 0s - loss: 0.0204 - acc: 0.9983
Epoch 00003: val_acc did not improve from 1.00000
2377/2377 [==============================]

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000002B0F4062318>
Traceback (most recent call last):
  File "E:\Anaconda3\envs\face_spoofing\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


In [ ]:
#kills the kernel to free-up memory on GPU, also avoiding collisions with othhers scripts 
#comment the line below, if you want to keep the variables and buffers
os._exit(00)